In [ ]:
import googlemaps
from datetime import datetime
import os
from dotenv import load_dotenv
from pathlib import Path
import pandas as pd

load_dotenv()
api_key = os.getenv("google_api")
gmaps = googlemaps.Client(key=api_key)
GDRIVE_FOLDER = Path(os.getenv("GDRIVE_FOLDER"))
origins = GDRIVE_FOLDER / "origin.csv"
destinations = GDRIVE_FOLDER / "attractions.csv"
origin_df = pd.read_csv(origins)
destinations_df = pd.read_csv(destinations)
now = datetime.now()


origin = (
    39.946196,
    -75.139832,
)  # hardcoded here since my origin doesn't change, but if yours does you can tweak the function to accept origins.
mode = "driving"  # accepts driving, walking, transit, cycling


def distance_duration_iteration():
    driving_list = []
    transit_list = []
    def distance_duration(destination):
        output = gmaps.directions(
            origin,
            destination,
            mode,
            units="imperial",
            departure_time=now,
        )
    
        if mode == "driving":
            driving_list.append(output)

        if mode == "transit":
            transit_list.append(output)


    """loops through the destinations file and runs the distance duration function"""
    for idx, row in destinations_df.iterrows():
        lat = row["Latitude"]
        lon = row["Longitude"]
        destination = (lat, lon)
        distance_duration(destination)
        
    if mode == "driving":
        return driving_list
    if mode == "transit":
        return transit_list
        


def df_to_csv(df):
    df.to_csv(GDRIVE_FOLDER / f"{mode}_travel_times.csv", sep=",")



In [155]:

def unpack_dicts(list_of_dicts):
    """function to crack into the nested dictionary structure that the api returns"""
    labels = ["distance", "duration", "lat", "lng"]
    df = pd.DataFrame(columns=labels)

    for item in list_of_dicts:
        destination_lat = item[0]["legs"][0]["end_location"]["lat"]
        destination_lng = item[0]["legs"][0]["end_location"]["lng"]
        distance = item[0]["legs"][0]["distance"]["text"]
        duration = item[0]["legs"][0]["duration"]["text"]
        d = {
            "distance": [distance],
            "duration": [duration],
            "lat": [destination_lat],
            "lng": [destination_lng],
        }
        list_df = pd.DataFrame(data=d)
        df = pd.concat([list_df, df], axis=0, ignore_index=True)

    return df
    
unpack_dicts(distance_duration_iteration())

,distance,duration,lat,lng
0,775 mi,11 hours 50 mins,33.791379,-84.389503
1,12.7 mi,21 mins,39.867424,-75.037018
2,10.1 mi,23 mins,39.897189,-75.031911
3,9.8 mi,22 mins,39.898873,-75.030261
4,5.6 mi,15 mins,39.920814,-75.119393
...,...,...,...,...
234,9.6 mi,17 mins,39.870684,-75.126822
235,21.4 mi,27 mins,39.724173,-75.172372
236,16.9 mi,25 mins,39.791171,-75.1513
237,13.6 mi,23 mins,39.835266,-75.155036
